# Optimizing a single point (n-dimensional pitch aggregate)

This noteboook demonstrates how to find the optimal tuning for a single point. In this case, a "point" is a pitch aggregate in n-dimensional space, where the pitch aggregate contains n+1 pitches. For example, a triad would be 2-dimensional. The values of the dimensions are equal to the base-2 logarithms of the ratios of the frequencies from one of the pitches (which we might call the "root").

## TensorBoard

If you would like to use [TensorBoard](https://www.tensorflow.org/tensorboard) to dig into the logs, you will need to install and run it from the command line. The following installation example uses conda, but you can use pip if you so desire.

```bash
conda install -n <your_environment> tensorboard
tensorboard --logdir logs/fit
```

## Setup and Usage

First, import all the necessary libraries.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf

devices = tf.config.experimental.get_visible_devices('GPU')
if len(devices) > 0:
    tf.config.experimental.set_memory_growth(devices[0], True)
    
import harmonic_distance as hd
import numpy as np
import datetime

## Global Constants

These constants should be scaled appropriately. Comments about each constant are given in-line.

In [3]:
# The "curve" of the parabola around each possible pitch. A higher 
# value will lead to fewer possible pitches.
C = 0.01

# The learning rate of the optimization algorithm. A higher value 
# will converge more quickly, if possible, but might never converge.
LEARNING_RATE = 1.0e-2

# The convergence threshold is the norm of the gradients of the loss
# function. This is used to test whether a proper "valley" has been found.
CONVERGENCE_THRESHOLD = 1.0e-5

# Maximum number of iterations before giving up on convergence.
MAX_ITERS = 1000

# These are logarithmic, so we're starting by trying to tune the 12TET major third to 5/4
STARTING_VALUES = [4.0 / 12.0]

# Dimensionality of the space. This is only finding the minimum of a single interval.
DIMENSIONS=1

## About the VectorSpace

The _vector space_ can be thought of as the list of "all possible pitches."

In this implementation, we're using a `VectorSpace` subclass of [`tf.Module`](https://www.tensorflow.org/api_docs/python/tf/Module?version=stable) to cache the variables that do not change through each iteration.

### `hd.vectors.space_graph_altered_permutations`

The array `[5, 5, 3, 3, 2, 1]` sets the number of degrees along each dimension of the harmonic lattice (Tenney 19XX) that are available for tuning. The `bounds` value of `(0.0, 4.0)` restricts these harmonic possibilities to a 4-octave range in pitch space.

### `hd.tenney.hd_aggregate_graph`

Calculates the harmonic distance of every vector.

### `vectors_reasonable`

Restricts the possible pitches to values with a harmonic distances less than `9.0`. This is useful for the purposes of reducing memory consumption.

In [4]:
def timestamped_writer(var=''):
    return tf.summary.create_file_writer('logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + var)

In [5]:
class Minimizer(tf.Module):
    def __init__(self, dimensions=1, **kwargs):
        self.dimensions = dimensions
        self.vs = hd.vectors.VectorSpace(**kwargs)
        self.log_pitches = tf.Variable(tf.zeros((dimensions), dtype=tf.float64), dtype=tf.float64)
        self.step = tf.Variable(0, dtype=tf.int64)
    
    def minimize(self):
        self.step.assign(0)
        self.writers = [timestamped_writer(var='/main')]
        for idx in range(self.dimensions):
            self.writers.append(timestamped_writer(var=("/pitch{}".format(idx+1))))
        # Since Adagrad maintains state, we need to reset it at the start of each call to minimize()
        self.opt = tf.optimizers.Adagrad(learning_rate=LEARNING_RATE)
        self.write_values()
        while self.stopping_op() and self.step < MAX_ITERS:
            self.opt.minimize(lambda: self.loss(self.log_pitches), self.log_pitches)
            self.step.assign_add(1)
            self.write_values()
        if self.stopping_op():
            tf.print("Did not converge")
            with self.writers[0].as_default():
                tf.summary.text("convergence", "did not converge", step=self.step)
        else:
            with self.writers[0].as_default():
                tf.summary.text("convergence", "converged", step=self.step)

    def write_values(self):
        current_loss = self.loss(self.log_pitches)
        with self.writers[0].as_default():
            tf.summary.scalar("loss", tf.reduce_mean(current_loss, axis=-1), step=self.step)
            with tf.GradientTape() as g:
                dz_dv = g.gradient(self.loss(self.log_pitches), self.log_pitches)
            norms = tf.nn.l2_loss(dz_dv)
            tf.summary.scalar("loss-norm", norms, step=self.step)
        for idx, writer in enumerate(self.writers[1:]):
            with writer.as_default():
                tf.summary.scalar("loss", current_loss[idx], step=self.step)
                tf.summary.scalar("pitch", self.log_pitches[idx], step=self.step)

    @tf.function
    def loss(self, var_list):
        return hd.optimize.parabolic_loss_function(self.vs.pds, self.vs.hds, var_list, curves=(C, C))
            
    @tf.function
    def stopping_op(self):
        with tf.GradientTape() as g:
            dz_dv = g.gradient(self.loss(self.log_pitches), self.log_pitches)
        norms = tf.nn.l2_loss(dz_dv)
        return norms >= CONVERGENCE_THRESHOLD

## Minimizing one interval

When we set `dimensions=1`, we are only attempting to minimize the harmonic distance of a single interval using the Adagrad (Adaptive Gradient Descent) algorithm.

## `prime_limits`

The `prime_limits` variable sets the maximum number of dimensions along the Tenney "harmonic lattice" that are used when generating the possible vector space. The defaults are overwritten here with smaller values (and restricting to a 7-limit harmonic space) to facilitate faster computation.

### About the Adagrad optimization algorithm

The Adagrad algorithm is short for "Adaptive Gradient Descent," and is implemented as part of the Tensorflow package. The algorithm uses a different learning rate for each dimension in the training variable vector (equivalent to a single "feature" in most machine learning applications). Variable that change by a large amount have their learning rates increased, while variables that change very little have very low learning rates. The advantage to this method is that it converges much more quickly than traditional gradient descent.

### Other algorithms

The Adam (Adagrad with Momentum) algorithm was also tried extensively. The "momentum" feature has the advantage for most applications of ensuring that the function does not fall into small local minima; however, for our purpose, we are very interested in finding local minima.

In [57]:
# working with a smaller set of prime limits here for speed of debugging
minimizer = Minimizer(dimensions=1, prime_limits=[4, 3, 2, 1])

### Running the algorithm

After initializing the `Minimizer`, we can assign the starting pitches as a one-dimensional `Tensor` or `Array` of shape `(dims,)`, where `dims` is equal to the `dimensions` argument from the initialization of the `Minimizer`.

Then, a single call to `minimizer.minimize()` will run the loop up to `MAX_ITERS` times until convergence is reached.

In [58]:
minimizer.log_pitches.assign([4/12])
minimizer.minimize()
minimizer.log_pitches

<tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([0.32192849])>

### Ration

In [63]:
winner = hd.vectors.closest_from_log([minimizer.log_pitches], minimizer.vs.vectors)
hd.vectors.to_ratio(winner[0])

(5.0, 4.0)